03717357 Gregory Ollivierre COMP6040 Assignment2 Task A

In [7]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[2] pyspark-shell"
os.environ['JAVA_HOME'] = "/home/gregory/jdk1.8.0_161/"
import findspark
spark_dir = '/home/gregory/spark-2.3.0-bin-hadoop2.7/'
findspark.init(spark_dir)
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [8]:
#Convert the following sentence into a python tuple list of letters and the frequency of which each letter appears in the current word.
#Ignore all non-alpha numeric characters.
sentence = "The quick brown fox jumps over the laxy dog and the fox was very happy"
# I treated 'T' and 't' as two seperate chars since they are different. Alternatively sentence.tolower()
# could be used
samp_tup_list = [(x,sentence.count(x)) for x in set(sentence.replace(" ",''))]
samp_tup_list

[('r', 3),
 ('u', 2),
 ('w', 2),
 ('j', 1),
 ('b', 1),
 ('g', 1),
 ('n', 2),
 ('c', 1),
 ('d', 2),
 ('o', 5),
 ('y', 3),
 ('v', 2),
 ('s', 2),
 ('m', 1),
 ('t', 2),
 ('l', 1),
 ('k', 1),
 ('h', 4),
 ('p', 3),
 ('T', 1),
 ('x', 3),
 ('f', 2),
 ('q', 1),
 ('a', 4),
 ('e', 5),
 ('i', 1)]

In [9]:
# create pyspark contect
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc = spark.sparkContext

In [10]:
# convert list of tuples to rdd
rdd = sc.parallelize(samp_tup_list)

In [11]:
# display letter count
rdd.collect()

[('r', 3),
 ('u', 2),
 ('w', 2),
 ('j', 1),
 ('b', 1),
 ('g', 1),
 ('n', 2),
 ('c', 1),
 ('d', 2),
 ('o', 5),
 ('y', 3),
 ('v', 2),
 ('s', 2),
 ('m', 1),
 ('t', 2),
 ('l', 1),
 ('k', 1),
 ('h', 4),
 ('p', 3),
 ('T', 1),
 ('x', 3),
 ('f', 2),
 ('q', 1),
 ('a', 4),
 ('e', 5),
 ('i', 1)]

In [12]:
# display the letter and number of times they appear in each word in the sentence
rdd.map(lambda x : (x[0],[y.count(x[0]) for y in sentence.split() if x[0] in y])).collect()

[('r', [1, 1, 1]),
 ('u', [1, 1]),
 ('w', [1, 1]),
 ('j', [1]),
 ('b', [1]),
 ('g', [1]),
 ('n', [1, 1]),
 ('c', [1]),
 ('d', [1, 1]),
 ('o', [1, 1, 1, 1, 1]),
 ('y', [1, 1, 1]),
 ('v', [1, 1]),
 ('s', [1, 1]),
 ('m', [1]),
 ('t', [1, 1]),
 ('l', [1]),
 ('k', [1]),
 ('h', [1, 1, 1, 1]),
 ('p', [1, 2]),
 ('T', [1]),
 ('x', [1, 1, 1]),
 ('f', [1, 1]),
 ('q', [1]),
 ('a', [1, 1, 1, 1]),
 ('e', [1, 1, 1, 1, 1]),
 ('i', [1])]

In [13]:
# create new pyspark contect
spark2 = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc2 = spark2.sparkContext

In [15]:
#lets open the csv and save to rdd
rdd2 = sc2.textFile('amazon_reviews.csv')
rdd2.take(5)

[',id,name,username',
 '0,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Cristina M',
 '1,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Ricky',
 '2,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Tedd Gardiner',
 '3,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Dougal']

In [16]:
#looks like comma seperated data. Lets split by ',' and remove first line since we add schema later
rdd3 = rdd2.map(lambda line: line.split(','))
rdd3 = rdd3.filter(lambda line : line[0] != '')
rdd3.take(5)

[['0', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Cristina M'],
 ['1', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Ricky'],
 ['2', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Tedd Gardiner'],
 ['3', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Dougal'],
 ['4', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Miljan David Tanic']]

In [17]:
# create dataframe with schema (index,idn,product,username)
df = rdd3.map(lambda line: Row(index=line[0],idn=line[1],product=line[2],username=line[3])).toDF()
df = df.drop('index') # drop index since its prob nor required
df.show()

+--------------------+-----------------+--------------------+
|                 idn|          product|            username|
+--------------------+-----------------+--------------------+
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|          Cristina M|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|               Ricky|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|       Tedd Gardiner|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|              Dougal|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|  Miljan David Tanic|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|          Kelvin Law|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|               Ricky|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|             Bandler|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|          Cristina M|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|       Tedd Gardiner|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|     Miguel Martinez|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|   Magnus Brattemark|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|       Tedd Gardiner|
|AVpe7As

In [18]:
#display top 20 bought products
df.groupby('product').count().sort('count',ascending=False).show()

+--------------------+-----+
|             product|count|
+--------------------+-----+
|Amazon Tap - Alex...|  542|
|      Amazon Fire TV|  166|
|Amazon Premium He...|  133|
|    Fire HD 6 Tablet|   87|
|"Kindle Fire HDX ...|   53|
|"Kindle Fire HDX ...|   43|
|"Kindle Fire HD 7"""|   41|
|   Kindle Paperwhite|   39|
|Certified Refurbi...|   38|
|     Kindle Keyboard|   32|
|All-New Amazon Fi...|   27|
|              Kindle|   20|
|Amazon 5W USB Off...|   19|
|All-New Amazon Fi...|   18|
|Replacement Remot...|   17|
|Echo Dot (2nd Gen...|   13|
|All-New Amazon Ki...|   13|
|Moshi Anti-Glare ...|   12|
|All-New Amazon Ki...|   12|
|Alexa Voice Remot...|   12|
+--------------------+-----+
only showing top 20 rows



In [19]:
#display top 20 users and what they bought
df.groupby('username','product').count().sort('count',ascending=False).show()

+-------------------+--------------------+-----+
|           username|             product|count|
+-------------------+--------------------+-----+
|          A. Younan|Amazon Premium He...|   59|
|             Andrew|Amazon Premium He...|   43|
|          Victor L.|Amazon Premium He...|   30|
|     William Hardin|    Fire HD 6 Tablet|   30|
|            Mike W.|    Fire HD 6 Tablet|   29|
|      Earthling1984|    Fire HD 6 Tablet|   28|
|     William Hardin|      Amazon Fire TV|   16|
|         B. Tarbuck|"Kindle Fire HDX ...|   16|
|              Mandy|      Amazon Fire TV|   16|
|                 NF|"Kindle Fire HDX ...|   15|
|                 NF|"Kindle Fire HD 7"""|   14|
|    Amazon Reviewer|"Kindle Fire HDX ...|   14|
|    Amazon Reviewer|"Kindle Fire HDX ...|   13|
|             Dallas|      Amazon Fire TV|   12|
|     William Hardin|Certified Refurbi...|   12|
|  Michael Gallagher|"Kindle Fire HDX ...|   12|
|   Gregory P. Baker|      Amazon Fire TV|   11|
|              MarkM

In [29]:
#Create a RDD of tuples from the dataframe from question 3 with only 2 columns ‘product’ and ‘username’ in that order.
rdd4 = df.select('product','username').rdd.map(tuple)
rdd4.take(10)

[('Kindle Paperwhite', 'Cristina M'),
 ('Kindle Paperwhite', 'Ricky'),
 ('Kindle Paperwhite', 'Tedd Gardiner'),
 ('Kindle Paperwhite', 'Dougal'),
 ('Kindle Paperwhite', 'Miljan David Tanic'),
 ('Kindle Paperwhite', 'Kelvin Law'),
 ('Kindle Paperwhite', 'Ricky'),
 ('Kindle Paperwhite', 'Bandler'),
 ('Kindle Paperwhite', 'Cristina M'),
 ('Kindle Paperwhite', 'Tedd Gardiner')]